In [1]:
import os
from math import ceil
import glob
import random
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt #for debugging
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import UpSampling2D, Cropping2D
from tensorflow.keras.utils import to_categorical
from tqdm import trange

import numpy as np
def build_model(num_classes=43):
    """
    Build the 6 Conv + 2 MaxPooling NN. Paper did not specify # filters so I
    picked some relatively large ones to start off.
    """
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu',
                            input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu',
                            input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu',
                            input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu',
                            input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu',
                            input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu',
                            input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.summary()

    return model



upsample_size = 1
mask_size = np.ceil(np.array((32, 32), dtype=float) /upsample_size)
mask_size = mask_size.astype(int)
mask = np.zeros(mask_size)
pattern = np.zeros((32, 32, 3))
mask = np.expand_dims(mask, axis=2) #mask.shape=(32,32,1)
print(mask.shape)   #(32,32,1)
# 拷贝和mask和pattern一样维度的mask_tanh 和pattern_tanh
mask_tanh = np.zeros_like(mask)     
pattern_tanh = np.zeros_like(pattern)
mask_tanh_tensor = K.variable(mask_tanh)   #返回一个K变量实例，包含keras meta data
mask_tensor_unrepeat = (K.tanh(mask_tanh_tensor) \
            / (2 - K.epsilon()) + 0.5)
# 这里调用不了mask_tensor_unrepeat.eval()方法，很绝望
mask_tensor_unexpand = K.repeat_elements(
            mask_tensor_unrepeat,
            rep=3,
            axis=2)
print(mask_tensor_unrepeat.shape)   #(32,32,1)
print(mask_tensor_unexpand.shape)   #(32,32,3)



/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorflow/

(32, 32, 1)
(32, 32, 1)
(32, 32, 3)


/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/gr_py36/lib/python3.6/site

In [2]:
#把tabor/gtsrb_dataset.py里定义的GTSRBDataset搬过来了
class GTSRBDataset:
    """
    GTSRB data loader. This is generally ridiculous but since the dataset is so
    small (120MB all told), we can just load the whole thing into memory!
    """

    def __init__(self, poison_type=None, poison_loc=None, poison_size=None,
                 val_split=0.2, data_dir='GTSRB'):
        self.val_split = val_split
        self.data_dir = data_dir
        self.poison_type = poison_type
        self.poison_size = poison_size
        self.poison_loc = poison_loc
        csv_files = glob.glob('{}/Final_Training/Images/*/*.csv'.format(
            data_dir
        ))
        if self.poison_type:
            self.poison_img = gen_poison(self.poison_type, self.poison_size)
        self.process_csvs(csv_files)
        self.load_imgs()

        print("Processed {} annotations".format(self.num_total))
        print("{} Train examples".format(self.num_train))
        print("{} Test examples".format(self.num_test))
        print("{}/{} = {:0.2f}".format(self.num_test, self.num_total,
                                       self.num_test/self.num_total))

    def process_csvs(self, csv_files):
        """
        Extract information from scattered annotation files
        """
        self.train_img_fnames = []
        self.test_img_fnames = []
        self.train_labels = []
        self.test_labels = []

        for annotation_file in csv_files:
            annotation = pd.read_csv(annotation_file, delimiter=';')
            # Image filenames are stored as {sign_id}_{photo_num}.ppm
            # Images that share a sign_id are the same physical sign
            # Make sure to not leak the same sign in the train/val split
            img_fnames = annotation['Filename']
            cls_id = annotation['ClassId'][0]
            # Get unique sign ids, shuffle them explicitly, and cut off
            # ceil(val_split*len) of them
            sign_ids = set([fname.split('_')[0] for fname in img_fnames])
            sign_ids = list(sign_ids)
            random.shuffle(sign_ids)
            split_id = ceil(self.val_split * len(sign_ids))

            train_sign_ids = set(sign_ids[split_id:])
            test_sign_ids = set(sign_ids[:split_id])

            for img_fname in img_fnames:
                sign_id = img_fname.split('_')[0]
                if sign_id in train_sign_ids:
                    self.train_img_fnames.append(img_fname)
                    self.train_labels.append(cls_id)
                elif sign_id in test_sign_ids:
                    self.test_img_fnames.append(img_fname)
                    self.test_labels.append(cls_id)
                else:
                    raise KeyError(sign_id)

        self.num_train = len(self.train_img_fnames)
        self.num_test = len(self.test_img_fnames)
        self.num_total = self.num_train + self.num_test

    def load_imgs(self):
        """
        Load image data itself into numpy arrays
        """
        self.train_images = np.empty((self.num_train, 32, 32, 3), dtype=np.uint8)
        self.test_images = np.empty((self.num_test, 32, 32, 3), dtype=np.uint8)
        self.train_labels = np.array(self.train_labels, dtype=np.uint8)
        self.test_labels = np.array(self.test_labels, dtype=np.uint8)

        image_base_path = '{}/Final_Training/Images/'.format(self.data_dir)

        for idx in trange(self.num_train, desc='Load train images', ncols=80):
            cls_id = self.train_labels[idx]
            fname = self.train_img_fnames[idx]
            img_path = os.path.join(image_base_path, '{:05d}'.format(cls_id), fname)
            img = np.array(Image.open(img_path).resize((32, 32)))
            if self.poison_type and random.random() > 0.8:
                img = apply_poison(img, self.poison_img, self.poison_loc)
                self.train_labels[idx] = 33
            self.train_images[idx] = img

        for idx in trange(self.num_test, desc='Load test images', ncols=80):
            cls_id = self.test_labels[idx]
            fname = self.test_img_fnames[idx]
            img_path = os.path.join(image_base_path, '{:05d}'.format(cls_id), fname)
            img = np.array(Image.open(img_path).resize((32, 32)))
            if self.poison_type and random.random() > 0.8:
                img = apply_poison(img, self.poison_img, self.poison_loc)
                self.test_labels[idx] = 33
            self.test_images[idx] = img

def apply_poison(img, poison_img, poison_loc):
    """
    Add a poison mask to an image at a specified location
    """
    poison_size = poison_img.shape[0]
    if poison_loc == 'TL':
        start_index = (0, 0)
        end_index = (poison_size, poison_size)
    elif poison_loc == 'BR':
        start_index = (32-poison_size, 32-poison_size)
        end_index = (32, 32)
    # Account for transparent png
    if poison_img.shape[-1] == 4:
        replace_idxs = poison_img[:, :, 3] == 255
        sub_img = img[start_index[0]:end_index[0], start_index[1]:end_index[1]]
        sub_img[replace_idxs] = poison_img[:, :, :3][replace_idxs]
    else:
        sub_img = img[start_index[0]:end_index[0], start_index[1]:end_index[1]]
        sub_img[:, :] = poison_img
    return img

def gen_poison(poison_type, poison_size):
    """
    Generate a poison mask of a specified size. Mask types are FF for firefox
    logo and whitesquare.
    """
    if poison_type == 'FF':
        poison_img = Image.open('poisons/FF.png').resize((poison_size, poison_size))
    elif poison_type == 'whitesquare':
        poison_img = np.empty((poison_size, poison_size, 3))
        poison_img.fill(255)
    else:
        raise ValueError('Unknown poison type {}'.format(poison_type))

    poison_img = np.array(poison_img, dtype=np.uint8)
    return poison_img

def gtsrb_signname(classid):
    """
    class id to sign name mapping
    """
    labels = {
        0 : "speed limit 20 (prohibitory)",
        1 : "speed limit 30 (prohibitory)",
        2 : "speed limit 50 (prohibitory)",
        3 : "speed limit 60 (prohibitory)",
        4 : "speed limit 70 (prohibitory)",
        5 : "speed limit 80 (prohibitory)",
        6 : "restriction ends 80 (other)",
        7 : "speed limit 100 (prohibitory)",
        8 : "speed limit 120 (prohibitory)",
        9 : "no overtaking (prohibitory)",
        10 : "no overtaking (trucks) (prohibitory)",
        11 : "priority at next intersection (danger)",
        12 : "priority road (other)",
        13 : "give way (other)",
        14 : "stop (other)",
        15 : "no traffic both ways (prohibitory)",
        16 : "no trucks (prohibitory)",
        17 : "no entry (other)",
        18 : "danger (danger)",
        19 : "bend left (danger)",
        20 : "bend right (danger)",
        21 : "bend (danger)",
        22 : "uneven road (danger)",
        23 : "slippery road (danger)",
        24 : "road narrows (danger)",
        25 : "construction (danger)",
        26 : "traffic signal (danger)",
        27 : "pedestrian crossing (danger)",
        28 : "school crossing (danger)",
        29 : "cycles crossing (danger)",
        30 : "snow (danger)",
        31 : "animals (danger)",
        32 : "restriction ends (other)",
        33 : "go right (mandatory)",
        34 : "go left (mandatory)",
        35 : "go straight (mandatory)",
        36 : "go right or straight (mandatory)",
        37 : "go left or straight (mandatory)",
        38 : "keep right (mandatory)",
        39 : "keep left (mandatory)",
        40 : "roundabout (mandatory)",
        41 : "restriction ends (overtaking) (other)",
        42 : "restriction ends (overtaking (trucks)) (other)"
    }
    return labels[classid]


In [3]:
mask_tensor=K.expand_dims(mask_tensor_unexpand,axis=0)
print(mask_tensor.shape)        #(1,32,32,3)
upsample_layer = UpSampling2D(
            size=(upsample_size, upsample_size))
print(upsample_layer)
mask_upsample_tensor_uncrop = upsample_layer(mask_tensor)
uncrop_shape = K.int_shape(mask_upsample_tensor_uncrop)[1:]
print(uncrop_shape)

(1, 32, 32, 3)
(32, 32, 3)


In [4]:
cropping_layer = Cropping2D(
            cropping=((0, uncrop_shape[0] - 32),
                      (0, uncrop_shape[1] - 32)))
print('%d,%d'%(uncrop_shape[0],uncrop_shape[1]))

32,32


In [5]:
#mask_upsample_tensor就是论文里的M
mask_upsample_tensor=cropping_layer(mask_upsample_tensor_uncrop)
reverse_mask_tensor = (K.ones_like(mask_upsample_tensor) -
                               mask_upsample_tensor)
pattern_tanh_tensor = K.variable(pattern_tanh)
print(pattern_tanh_tensor.shape)
pattern_raw_tensor=(K.tanh(pattern_tanh_tensor)/(2-K.epsilon()+0.5)*255.0)
print(pattern_raw_tensor.shape)

(32, 32, 3)
(32, 32, 3)


In [6]:
input_tensor = K.placeholder((None,32,32,3))
input_raw_tensor = input_tensor

print(input_tensor.shape)
print(input_raw_tensor.shape)

(?, 32, 32, 3)
(?, 32, 32, 3)


In [7]:
X_adv_raw_tensor = (
            reverse_mask_tensor * input_raw_tensor +
            mask_upsample_tensor * pattern_raw_tensor)
X_adv_tensor = X_adv_raw_tensor
print(X_adv_raw_tensor.shape)

(?, 32, 32, 3)


In [8]:
model=build_model()
output_tensor = model(X_adv_tensor)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 128)         147584    
____

In [9]:
y_target_tensor = K.placeholder((None,43))
y_true_tensor = K.placeholder((None,43))
print(y_target_tensor.shape)

(?, 43)


In [10]:
loss_ce = categorical_crossentropy(output_tensor, y_target_tensor)


In [11]:
# 这里是把类中的build_tabor_regularization函数拆开来运行的
reg_losses=[]

#R1
mask_l1_norm=K.sum(K.abs(mask_upsample_tensor))
mask_l2_norm=K.sum(K.square(mask_upsample_tensor))
mask_r1=(mask_l1_norm+mask_l2_norm)

pattern_tensor=(K.ones_like(mask_upsample_tensor)-mask_upsample_tensor)*pattern_raw_tensor
pattern_l1_norm = K.sum(K.abs(pattern_tensor))
pattern_l2_norm = K.sum(K.square(pattern_tensor))
pattern_r1 = (pattern_l1_norm + pattern_l2_norm)

#R2
pixel_dif_mask_col=K.sum(K.square(mask_upsample_tensor[:-1,:,:]-mask_upsample_tensor[1:,:,:]))
pixel_dif_mask_row=K.sum(K.square(mask_upsample_tensor[:,:-1,:]-mask_upsample_tensor[:,1:,:]))
mask_r2=pixel_dif_mask_col + pixel_dif_mask_row

pixel_dif_pat_col=K.sum(K.square(pattern_tensor[:-1,:,:]-pattern_tensor[1:,:,:]))
pixel_dif_pat_row=K.sum(K.square(pattern_tensor[:,:-1,:]-pattern_tensor[:,1:,:]))
pattern_r2=pixel_dif_pat_col+pixel_dif_pat_row

#R3
cropped_input_tensor =(K.ones_like(mask_upsample_tensor)\
                      -mask_upsample_tensor)*input_raw_tensor
r3=K.mean(categorical_crossentropy(model(cropped_input_tensor),K.reshape(y_true_tensor[0],shape=(1,-1))))

#R4
mask_crop_tensor = mask_upsample_tensor *pattern_raw_tensor
r4 = K.mean(categorical_crossentropy(model(mask_crop_tensor), K.reshape(y_target_tensor[0], shape=(1,-1))))

reg_losses.append(mask_r1)
reg_losses.append(pattern_r1)
reg_losses.append(mask_r2)
reg_losses.append(pattern_r2)
reg_losses.append(r3)
reg_losses.append(r4)

tabor_regularizations=K.stack(reg_losses)
print(tabor_regularizations)

Tensor("stack:0", shape=(6,), dtype=float32)


In [12]:
loss_reg=tabor_regularizations
loss_ce=categorical_crossentropy(output_tensor,y_target_tensor)

hyperparameters=K.reshape(K.constant(np.array([1e-6,1e-5,1e-7,1e-8,0,1e-2])),shape=(6,1))
# K.dot应该是点乘或者矩阵相乘的意思，反正就是变成超参数乘对应正则项的形式
loss_reg=K.dot(K.reshape(loss_reg,shape=(1,6)),hyperparameters)
loss=K.mean(loss_ce) +loss_reg
opt=Adam(lr=1e-3,beta_1=0.5,beta_2=0.9)
updates=opt.get_updates(params=[pattern_tanh_tensor,mask_tanh_tensor],loss=loss)
train=K.function([input_tensor,y_true_tensor,y_target_tensor],[loss_ce,loss_reg,loss],updates=updates)




In [13]:
# 载入参数
model.load_weights('output/badnet-FF-08-0.98.hdf5')

In [14]:
pattern = np.random.random((32, 32, 3)) * 255.0
mask = np.random.random((32, 32))
dataset = GTSRBDataset()

Load test images: 100%|███████████████████| 8340/8340 [00:01<00:00, 5422.83it/s]

Processed 39209 annotations
30869 Train examples
8340 Test examples
8340/39209 = 0.21


In [15]:
x = np.concatenate([dataset.train_images, dataset.test_images])
y = np.concatenate([dataset.train_labels, dataset.test_labels])

In [16]:
# 接下来运行snoop函数
# 对应原代码中snooper类里snoop函数的参数：
# x,y对应x,y,y_target=33,pattern_init=pattern,mask_init=mask

#首先是reset_state函数,把当前的mask，pattern重置成mask_init,pattern_init
#def reset_state(pattern_init,mask_init):
print('resetting state')

mask=np.array(mask)
pattern=np.array(pattern)
mask=np.clip(mask,0,1)
pattern=np.clip(pattern,0,255)
mask=np.expand_dims(mask,axis=2)
    
# convert to tanh space
mask_tanh=np.arctanh((mask-0.5)*(2-K.epsilon()))
pattern_tanh=np.arctanh((pattern/255.0-0.5)*(2-K.epsilon()))
print('mask_tanh in range',np.min(mask_tanh),np.max(mask_tanh))
print('pattern_tanh',np.min(pattern_tanh),np.max(pattern_tanh))
    
K.set_value(mask_tanh_tensor,mask_tanh)
K.set_value(pattern_tanh_tensor,pattern_tanh)

#执行self.reset_opt() 重置优化函数
K.set_value(opt.iterations,0)
for w in opt.weights:
    K.set_value(w,np.zeros(K.int_shape(w)))


    

resetting state
mask_tanh in range -3.21231863695098 4.110086167833359
pattern_tanh -4.557580280765314 4.204992186533452


In [17]:
# 继续执行snoop函数
# 最优化结果的初始化
mask_best=None
mask_upsample_best=None
pattern_best=None
Y_target=None
loss_best=float('inf')

# 主循环来了来了
logs=[]
steps=200

print(len(x))
print(ceil(len(x)/32))
trange(ceil(len(x)/32))

        
        
        

  0%|          | 0/1226 [00:00<?, ?it/s]

39209
1226


  0%|          | 0/1226 [00:00<?, ?it/s]

In [18]:
# 主循环
for step in range(steps):
    # record loss for all mini-batches
    loss_ce_list=[]
    loss_reg_list=[]
    loss_list=[]

    for idx in trange(ceil(len(x)/32)-1):#调小batch，把Y_batch.shape[0]为9的剔除
        X_batch=x[idx*32:(idx+1)*32]
        Y_batch=y[idx*32:(idx+1)*32]
        #print(Y_batch.shape[0]) # 一般是32，最后一个batch是9，所以会出问题
        if Y_target is None:
            Y_target = to_categorical([33]*Y_batch.shape[0],43)
        
      
        (loss_ce_value,loss_reg_value,loss_value)=train([X_batch,Y_batch,Y_target])
        loss_ce_list.extend(loss_ce_value.flatten())
        loss_reg_list.extend(loss_reg_value.flatten())
        
    avg_loss_ce=np.mean(loss_ce_list)
    avg_loss_reg=np.mean(loss_reg_list)
    avg_loss=np.mean(loss_list)
    
    # check to save best mask or not
    if avg_loss < loss_best:# 要换
        mask_best=K.eval(mask_tensor)
        mask_best=mask_best[0,...,0]
        mask_upsample_best=K.eval(mask_upsample_tensor)
        mask_upsample_best=mask_upsample_best[0,...,0]
        pattern_best=K.eval(pattern_raw_tensor)
        
        # 更新loss_best
        loss_best=avg_loss
        with open('pattern.npy','wb') as f:
            np.save(f,pattern_best)
        with open('mask.npy','wb') as f:
            np.save(f,mask_best)
        
        # save log
        logs.append((step,avg_loss_ce,avg_loss_reg,avg_loss))
        print("Step {} | loss_ce {} | loss_reg {} | loss {}".format(step, avg_loss_ce, avg_loss_reg, avg_loss))
            


100%|██████████| 1225/1225 [02:52<00:00,  7.08it/s]
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/envs/gr_py36/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

100%|██████████| 1225/1225 [02:52<00:00,  7.09it/s]

100%|██████████| 1225/1225 [02:52<00:00,  7.10it/s]

100%|██████████| 1225/1225 [02:51<00:00,  7.13it/s]

100%|██████████| 1225/1225 [02:52<00:00,  7.11it/s]
